# let's build an CNN with pytorch 
I will build an cnn and apply data augmentation  and batch normalization 
## What's cnn?
- CCNs are probably the most popular of all neural networ architecures. 

- This is mainly because, although they work in many domains, they are particularly good at dealing with images, and advances in technology have allowed the collection of large amounts of images to be possible in order to tackle a great variety of today's challenges.

- From image classification to object detection, CNNs are being used to diagnose cancer patients and detect fraud in systems, as well as to construct well thought-out self- driving vehicles that will revolutionize the future.
## Building Blocks of CNNs
A deep convolutional network is one that takes an image as an input, passes it through a series of convolutional layers with filters, pooling layers, and fully connected layers, to finally apply a softmax activation function that classifies the image into a class label. The classification, as with ANNs, is performed by calculating the probability of the image belonging to each of the class labels, giving each class label a value between zero and one. The class label with the higher probability is the one
selected as the final prediction for that image.
### Convolutional layers 
This is the first step to extract features from an image. The objective is to maintain the
relation between nearby pixels by learning the features over small sections of the image A mathematical operation occurs in this layer, where two inputs are given (the image
and the filter) and an output is obtained. As explained before, the operation consists
of convolving the filter and a section of the image of the same size of the filter. This
operation is repeated for all subsections of the image.
### Padding
The padding feature, as the name indicates, pads the image with zeros. This means that
it adds additional pixels to each side of the image, which are filled with zeros.
### Strdie
This parameter refers to the number of pixels that the filter will shift over the input
matrix, both horizontally and vertically. As we have seen so far, the filter is passed
through the top-left corner of the image, then it shifts over to the right by one pixel,
and so on until it has gone through all sections of the image vertically and horizontally.
This example is one of a convolutional layer, with stride equal to one, which is the
default configuration for this parameter.

# Data Augmentation 
In simple words, it is a measure to
increase the number of training examples by slightly modifying the existing examples.
For example, you could duplicate the instances currently available and add some noise
to those duplicates to make sure they are not exactly the same.
In computer vision problems, this means incrementing the number of images in the
training dataset by altering the existing images, which can be done by slightly altering
the current images to create duplicated versions that are slightly different
These minor adjustments to the images can be in the form of slight rotations, changes
in the position of the object in the frame, horizontal or vertical flips, different color
schemes, and distortions, among others. This technique works considering that CNNs
will consider each of these images a different image.

In [ ]:
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [ ]:
transform = {
    "train": transforms.Compose([
        transforms.RandomHorizontalFlip(0.5), 
        transforms.RandomGrayscale(0.1),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]),
    "test": transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])}

### Convert the pixels into tensor datatype and normalizing the data 

In [ ]:
batch_size = 100

train_data = datasets.CIFAR10('data3', train=True, download=True, transform=transform["train"])
test_data = datasets.CIFAR10('data3', train=False, download=True, transform=transform["test"])

batch_size = 100 
## downloading the data set from datasets module 
train_data = datasets.CIFAR10('data', train=True, download=True, transform=transform)
test_data = datasets.CIFAR10('data', train=False, download=True, transform=transform)

dev_size = 0.2 #Using a validation size of 20%, define the training and validation sampler that will be used to divide the dataset into those two sets.
idx = list(range(len(train_data)))
np.random.shuffle(idx)
split_size = int(np.floor(dev_size * len(train_data)))
train_idx, dev_idx = idx[split_size:], idx[:split_size]

train_sampler = SubsetRandomSampler(train_idx)
dev_sampler = SubsetRandomSampler(dev_idx)

- The SubsetRandomSampler() function from pytorch is used to divide the original training set into training and validations by randomly sampling indexes. 


In [ ]:
dev_size = 0.2
idx = list(range(len(train_data)))
np.random.shuffle(idx)
split_size = int(np.floor(dev_size * len(train_data)))
train_idx, dev_idx = idx[split_size:], idx[:split_size]

train_sampler = SubsetRandomSampler(train_idx)
dev_sampler = SubsetRandomSampler(dev_idx)

- The DataLoader() functions are the ones in charge of loading the images by batches.The resulting variables(train_loader,dev_loader,and test_loader) of this function will contain the values for the features and the target separately. 

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)
dev_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=dev_sampler)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

### Define the architecture of your network. Use the following information to do so:
• Conv1: A convolutional layer that takes as input the colored image and passes it
through 10 filters of size 3. Both the padding and the stride should be set to 1.

• Conv2: A convolutional layer that passes the input data through 20 filters of size 3.
Both the padding and the stride should be set to 1.

• Conv3: A convolutional layer that passes the input data through 40 filters of size
three. Both the padding and the stride should be set to 1.

• Use the ReLU activation function after each convolutional layer.

• A pooling layer after each convolutional layer, with a filter size and stride of 2.

• A dropout term set to 20% after flattening the image.

• Linear1: A fully-connected layer that receives as input the flattened matrix from
the previous layer and generates an output of 100 units. Use the ReLU activation
function for this layer. A dropout term here is set to 20%.

• Linear2: A fully-connected layer that generates 10 outputs, one for each class label.
Use the log_softmax activation function for the output layer.

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, 3, 1, 1)
        self.norm1 = nn.BatchNorm2d(10)
        self.conv2 = nn.Conv2d(10, 20, 3, 1, 1)
        self.norm2 = nn.BatchNorm2d(20)
        self.conv3 = nn.Conv2d(20, 40, 3, 1, 1)
        self.norm3 = nn.BatchNorm2d(40)
        self.pool = nn.MaxPool2d(2, 2)
        
        self.linear1 = nn.Linear(40 * 4 * 4, 100)
        self.norm4 = nn.BatchNorm1d(100)
        self.linear2 = nn.Linear(100, 10)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.pool(self.norm1(F.relu(self.conv1(x))))
        x = self.pool(self.norm2(F.relu(self.conv2(x))))
        x = self.pool(self.norm3(F.relu(self.conv3(x))))

        x = x.view(-1, 40 * 4 * 4)
        x = self.dropout(x)
        x = self.norm4(F.relu(self.linear1(x)))
        x = self.dropout(x)
        x = F.log_softmax(self.linear2(x), dim=1)
        
        return x

In [ ]:
model = CNN()
loss_function = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 100

- Train your network and be sure to save the values for the loss and accuracy of both the training and validation sets. 

In [ ]:
train_losses, dev_losses, train_acc, dev_acc= [], [], [], []
x_axis = []

for e in range(1, epochs+1):
    losses = 0
    acc = 0
    iterations = 0
    
    model.train()
    for data, target in train_loader:
        iterations += 1

        pred = model(data)
        loss = loss_function(pred, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        losses += loss.item()
        p = torch.exp(pred)
        top_p, top_class = p.topk(1, dim=1)
        acc += accuracy_score(target, top_class)
        
    dev_losss = 0
    dev_accs = 0
    iter_2 = 0
        
    if e%5 == 0 or e == 1:
        x_axis.append(e)
        
        with torch.no_grad():
            model.eval()
            
            for data_dev, target_dev in dev_loader:
                iter_2 += 1
                
                dev_pred = model(data_dev)
                dev_loss = loss_function(dev_pred, target_dev)
                dev_losss += dev_loss.item()

                dev_p = torch.exp(dev_pred)
                top_p, dev_top_class = dev_p.topk(1, dim=1)
                dev_accs += accuracy_score(target_dev, dev_top_class)
        
        train_losses.append(losses/iterations)
        dev_losses.append(dev_losss/iter_2)
        train_acc.append(acc/iterations)
        dev_acc.append(dev_accs/iter_2)
        
        print("Epoch: {}/{}.. ".format(e, epochs),
              "Training Loss: {:.3f}.. ".format(losses/iterations),
              "Validation Loss: {:.3f}.. ".format(dev_losss/iter_2),
              "Training Accuracy: {:.3f}.. ".format(acc/iterations),
              "Validation Accuracy: {:.3f}".format(dev_accs/iter_2))

- Plot the loss and accuracy of both sets

In [ ]:
plt.plot(x_axis,train_losses, label='Training loss')
plt.plot(x_axis, dev_losses, label='Validation loss')
plt.legend(frameon=False)
plt.show()

In [ ]:
plt.plot(x_axis, train_acc, label="Training accuracy")
plt.plot(x_axis, dev_acc, label="Validation accuracy")
plt.legend(frameon=False)
plt.show()

In [ ]:
model.eval()
iter_3 = 0
acc_test = 0
for data_test, target_test in test_loader:
    iter_3 += 1
    test_pred = model(data_test)
    test_pred = torch.exp(test_pred)
    top_p, top_class_test = test_pred.topk(1, dim=1)
    acc_test += accuracy_score(target_test, top_class_test)
print(acc_test/iter_3)